In [2]:
cd /content/Tich-Hop-Du-Lieu

/content/Tich-Hop-Du-Lieu


In [3]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy,datetime,re
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
tentative_engagements = []
free_bds = []
corresponacy1 = []

In [6]:
def matchingtablebyheaders(tbl1,tbl2):      
    # For matching
    Pref=np.zeros((len(tbl1),len(tbl2)))
        #assign the matching ratio between tbl1 and tbl2
    for i in range(len(tbl1)):
        for j in range(len(tbl2)):
            Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
    return Pref

In [7]:
def listofpreferencespertable(tbl1,tbl2,Pref):
        #for chotot
    ipref_tom=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
    #for bds
    ipref_bds=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
    pref_bds=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
    preferred_rankings_chotot = {}
    preferred_rankings_bds = {}
    #create list of preferred_rankings_chotot with header
    #assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl2 (17)
    for i in range(len(tbl2)):
        ipref_tom[:,i]=np.argsort(Pref[:,i])
        ipref_tom[:,i]=ipref_tom[::-1,i]
        preferred_rankings_chotot[i] = ipref_tom[:,i]
        #pref_tom[:,i]=np.sort(Pref[:,i])
        #pref_tom[:,i]=pref_tom[::-1,i]
    #ipref_tom

    #create list of preferred_rankings_bds with header
    #assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl1 (13)
    for i in range(len(tbl1)):
        #get indexs values of rows and store them in the rows as well
        ipref_bds[:,i] = np.argsort(Pref[i,:])
        ipref_bds[:,i] = ipref_bds[::-1,i]
        preferred_rankings_bds[i] = ipref_bds[:,i]
        #pref_bds[:,i] = np.sort(Pref[i,:])
      #  pref_bds[:,i] = pref_bds[::-1,i]
    #ipref_bds
    return preferred_rankings_bds,preferred_rankings_chotot

In [8]:
#Imdb which still need to propose and get accepted successfully
#Keep track of the headers that "may" end up together
def init_free_bds(preferred_rankings_imdb):
    for imdb in preferred_rankings_imdb.keys():
        free_bds.append(imdb)

In [9]:
def begin_matching(bds,preferred_rankings_bds,preferred_rankings_chotot):
    #  '''Find the first free Tomaten available to a bds at any given time'''
    #print(preferred_rankings_bds[bds])
    for tomate in preferred_rankings_bds[bds]:
        #Boolean for whether tomate is taken or not
        taken_match = [couple for couple in tentative_engagements if tomate == couple [:][1]]
        if (len(taken_match) == 0):
            #tentatively engage the man and woman
            tentative_engagements.append([bds, tomate])
            free_bds.remove(bds)
            break
        elif (len(taken_match) > 0):

            current_bds = list(preferred_rankings_chotot[tomate].tolist()).index(taken_match[0][0])
            potential_bds = list(preferred_rankings_chotot[tomate].tolist()).index(bds)

            if (current_bds < potential_bds):
                ii=[]
             
            else:         
                free_bds.remove(bds)
                #The old bds is now single
                free_bds.append(taken_match[0][0])
                #Update the fiance of the tomaten (tentatively)
                taken_match[0][0] = bds
                break

In [10]:
def stable_matching(preferred_rankings_bds,preferred_rankings_chotot):
    '''Matching algorithm until stable match terminates'''
    while (len(free_bds) > 0):
        for imdb in free_bds:
            begin_matching(imdb,preferred_rankings_bds,preferred_rankings_chotot)

In [11]:
def check_short_matching(COL1,COL2):
    rate = 0.0
    if str(COL1) == 'nan' or str(COL2) == 'nan' :
        return 0
    COL1 = str(COL1)
    COL2 = str(COL2)
    sw = (fuzz.SequenceMatcher(None,COL1, COL2).ratio())
    return sw #/(max(len(COL1),len(COL2))*1.0)


In [12]:
def check_match(st1,st2):
    str1 = np.array((st1.values).astype(str))
    str2 = np.array((st2.values).astype(str))
    # ret = np.zeros((len(str1),len(str2)))
    ret = np.zeros((len(str1)))
    
    for i in range(len(str1)):
        tmp = -1
        for j in range(len(str2)):
            # ret[i,j] = check_short_matching(str1[i],str2[j])
            tmp = max(tmp, check_short_matching(str1[i],str2[j]))
        ret[i] = tmp
    return numpy.average(ret)

In [13]:
def matchingtablebyvalues(BDS_DF_SAMPLE,CHOTOT_DF_SAMPLE,tbl1,tbl2):
    dd = []
    ret = []
    value1 = []
    value2 = []
    index_i = 0
    Pref=np.zeros((len(tbl1),len(tbl2)))
    for i in BDS_DF_SAMPLE:
        dd = []
        index_j = 0
        for j in CHOTOT_DF_SAMPLE:
            table1 = BDS_DF_SAMPLE[i]
            table2 = CHOTOT_DF_SAMPLE[j]
            value1 = (table1.values).astype(str)
            value2 = (table2.values).astype(str)
            #dd.append([ 100* (check_match(table1, table2)),i,j])
            Pref[index_i,index_j] = 100 * (np.float(check_match(table1, table2)))
            index_j+=1
        index_i+=1
    return Pref

In [14]:
def outputforvalues(tbl1,tbl2,Pref):
    i = 0
    for item in tentative_engagements:
        matching=Pref[item[0],item[1]]
        print(tbl1[item[0]] + " -----" ,tbl2[item[1]],':',matching)

BDS

In [15]:
batdongsan_origin = pd.read_csv("DIProject/crawler/Bat-dong-san2.csv")

In [16]:
batdongsan = pd.DataFrame(batdongsan_origin[['Original Link', 'Tiêu đề', 'Tên người bán', 'Số điện thoại', 'Địa chỉ', 'Mô tả', 'Mức giá', 'Diện tích', 'Ảnh', 'Hướng nhà', 'Số toilet', 'Số phòng ngủ', 'Pháp lý', 'Nội thất']])

In [17]:
# batdongsan = batdongsan.dropna(subset = ['Diện tích'])
batdongsan = batdongsan.dropna(subset = ['Ảnh'])

In [18]:
batdongsan

,Original Link,Tiêu đề,Tên người bán,Số điện thoại,Địa chỉ,Mô tả,Mức giá,Diện tích,Ảnh,Hướng nhà,Số toilet,Số phòng ngủ,Pháp lý,Nội thất
0,https://batdongsan.com.vn/ban-dat-xa-tan-lap-1...,Bán 10 mẫu ngay mặt tiền đường dẫn cao tốc Ph...,Quách Hương,0979016016,"Xã Tân Lập, Hàm Thuận Nam, Bình Thuận",\r\n Đất ngay đường dẫn cao tốc...,Thỏa thuận,93252 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN
1,https://batdongsan.com.vn/ban-dat-duong-quang-...,"Tôi không bán đất, tôi chỉ muốn khoe tôi có 2...",Quách Hương,0979016016,"Đường Quang Trung, Phường Tăng Nhơn Phú B, Quậ...",\r\n Sở hữu đất rộng ngay UBND ...,Thỏa thuận,238 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN
2,https://batdongsan.com.vn/ban-dat-duong-dt-716...,Tôi không bán Đất. Tôi chỉ muốn khoe tôi có 3...,Quách Hương,0979016016,"Đường ĐT 716, Xã Hoà Thắng, Bắc Bình, Bình Thu...",\r\n Siêu phẩm đất view biển nằ...,Thỏa thuận,4169 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN
3,https://batdongsan.com.vn/ban-dat-nen-du-an-du...,Hưng Thịnh mở bán 40 lô đất nền đô thị kiểu m...,Tuấn Trần,0901478123,"Dự án Richmond Quy Nhơn, Quy Nhơn, Bình Định",\r\n - Tập đoàn Hưng Thịnh mở b...,5.3 tỷ,100 m²,https://file4.batdongsan.com.vn/resize/200x200...,Đông,NaN,NaN,Hợp đồng mua bán,NaN
4,https://batdongsan.com.vn/ban-nha-biet-thu-lie...,"Quỹ 5 căn góc liền kề, biệt thự trục chính, m...",Đào Tiến Minh,0348666658,"Dự án Mê Linh - Thanh Lâm - Đại Thịnh, Mê Linh...",\r\n Chính chủ hiện nay tôi có ...,7.3 tỷ,111 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,5 phòng,4 phòng,NaN,Nhà xây thô hoàn thiện mặt ngoài.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,https://batdongsan.com.vn/ban-nha-biet-thu-lie...,"Bán gấp villa lô góc hồ bơi xịn xò Vĩnh Hiệp,...",Nguyen Truc Oanh,0905005028,"Xã Vĩnh Hiệp, Nha Trang, Khánh Hòa",\r\n Phù hợp cho khách hàng mua...,6.8 tỷ,261.5 m²,https://file4.batdongsan.com.vn/resize/200x200...,Đông,7 phòng,5 phòng,Sổ đỏ/ Sổ hồng,Nội thất đầy đủ và cao cấp
2371,https://batdongsan.com.vn/ban-dat-pho-bau-lach...,Chính chủ bán đất Củ Chi sát Vành Đai 4 Tp.HC...,Tran Van Hien,0866068796,"Phố Bàu Lách, Xã Nhuận Đức, Củ Chi, Hồ Chí Minh","\r\n Mặt tiền đường Bàu Lách, c...",2.2 tỷ,125 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,NaN,NaN,NaN,NaN
2372,https://batdongsan.com.vn/ban-dat-nen-du-an-ph...,Sở hữu ngay đất biệt thự khu biệt lập Euro Vi...,Nguyễn Văn M,0932560868,"Dự án Euro Village 2, Phường Hòa Xuân, Cẩm Lệ,...",\r\n Euro Village 2 - Nơi trải ...,Thỏa thuận,300 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,NaN,NaN,NaN,NaN
2373,https://batdongsan.com.vn/ban-can-ho-chung-cu-...,Siêu ưu đãi!Chỉ từ 550tr sở hữu ngay CHCC 2PN...,Xuân Mỹ,0344005887,"Dự án Lumiere Boulevard, Đường Nguyễn Xiển, Ph...","\r\n Rồi một ngày, bạn nhận ra,...",3.9 tỷ,67 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,2 phòng,2 phòng,NaN,Nội thất hoàn thiện cơ bản


In [19]:
batdongsan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2226 entries, 0 to 2374
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Original Link  2226 non-null   object
 1   Tiêu đề        2226 non-null   object
 2   Tên người bán  2226 non-null   object
 3   Số điện thoại  2226 non-null   object
 4   Địa chỉ        2226 non-null   object
 5   Mô tả          2226 non-null   object
 6   Mức giá        2226 non-null   object
 7   Diện tích      2225 non-null   object
 8   Ảnh            2226 non-null   object
 9   Hướng nhà      772 non-null    object
 10  Số toilet      1121 non-null   object
 11  Số phòng ngủ   1225 non-null   object
 12  Pháp lý        1440 non-null   object
 13  Nội thất       704 non-null    object
dtypes: object(14)
memory usage: 260.9+ KB


In [20]:
def normalize_price(x):   
    if 'tỷ' in x or 'triệu' in x:
      if (x[-2:]) == 'tỷ':
        x = str(float(x[:-3].replace(',', '.'))*1000)
      x = x.replace('tỷ','')
      x = x.replace('triệu','')  
      x = x.replace(' ', '')
      try:
        x = float(x)
        if x > 0:
          return x
        else: 
          return str('Thỏa thuận')
      except:
        return str('Thỏa thuận')
    else:
      return str('Thỏa thuận')

In [21]:
batdongsan['Mức giá'] = batdongsan['Mức giá'].apply(lambda x: normalize_price(str(x)))

In [22]:
def normalize_square(x):   
  x = x.split(' ')
  if x[0] != 'nan' and len(x[0]) > 0:

    x[0] = x[0].replace(',', '.')
    return float(x[0])

In [23]:
batdongsan['Diện tích'] = batdongsan['Diện tích'].apply(lambda x: normalize_square(str(x)))   

In [24]:
def normalize_string(x):   
  x = x.lower()
  x = x.encode('utf-8')
  return x

In [25]:
batdongsan['Tiêu đề'] = batdongsan['Tiêu đề'].apply(lambda x: normalize_string(str(x)))   
batdongsan['Địa chỉ'] = batdongsan['Địa chỉ'].apply(lambda x: normalize_string(str(x)))  
batdongsan['Mô tả'] = batdongsan['Mô tả'].apply(lambda x: normalize_string(str(x)))      

BDS & CHOTOT

In [26]:
chotot = pd.read_csv("DIProject/crawler/Cho-tot2.csv")

In [27]:
chotot.drop('Unnamed: 0', inplace=True, axis=1)

In [28]:
chotot

,Số phòng ngủ:,Chiều ngang:,Mã căn / Mã căn hộ:,Số phòng vệ sinh:,Chi tiết,Dự Án:,Link nguồn,Loại hình nhà ở:,Ảnh nhà,Tình trạng nội thất:,...,Liên lạc,Giá/m2:,Chiều dài:,Giá đề xuất,Diện tích sử dụng:,Địa chỉ,Tên bài đăng,Tên phân khu/Lô/Block/Tháp:,Tổng số tầng:,Hướng cửa chính:
0,2 phòng,4 m,NaN,1 phòng,"Nhà trệt lầu hoàn công, nở hậu, Hẻm 98 - Phạm ...",NaN,https://nha.chotot.com/mua-ban-nha-dat-quan-ni...,Nhà ngõ,https://cdn.chotot.com/K1dRMhyumnlHZmFwkweuO6F...,Hoàn thiện cơ bản,...,0943700368,"47,11 triệu/m²",10 m,"1,79 tỷ",NaN,"Đường Phạm Ngũ Lão, Phường An Hòa, Quận Ninh K...",Nhà trệt lầu hẻm 98 Phạm Ngũ Lão,NaN,1,Tây Bắc
1,3 phòng,NaN,NaN,NaN,Bán nhà Vân Canh 5 TẦNG MỚI Ở NGAY \r\n+Oto tr...,NaN,https://nha.chotot.com/mua-ban-nha-dat-huyen-h...,Nhà ngõ,https://cdn.chotot.com/qgN7JPtEr2_wkCN-NIpfnmE...,NaN,...,0376369183,75 triệu/m²,NaN,"2,4 tỷ",NaN,"Đường An Trai, Xã Vân Canh, Huyện Hoài Đức, Hà...",BÁN NHÀ VÂN CANH 5 TẦNG Ở NGAY CHỈ 2TY4,NaN,NaN,NaN
2,2 phòng,4 m,NaN,2 phòng,Chủ cần sang nhượng nhà để di cư sang nước ngo...,NaN,https://nha.chotot.com/mua-ban-nha-dat-quan-bi...,Nhà ngõ,https://cdn.chotot.com/16mWE6EtFcFUjq3SbgeMd1y...,NaN,...,0938455817,"26,96 triệu/m²",11.5 m,"1,24 tỷ",46 m²,"Đường Lê Văn Quới, Phường Bình Trị Đông A, Quậ...","NHÀ HẺM 1 TRỆT 1 LẦU, LÊ VĂN QUỚI_B.TÂN/46m2,...",NaN,1,NaN
3,2 phòng,NaN,NaN,2 phòng,"- Vị trí nhà đẹp, lô góc 2 mặt thoáng, căn nhà...",NaN,https://nha.chotot.com/mua-ban-nha-dat-quan-do...,Nhà ngõ,https://cdn.chotot.com/rJNmUj6bFC4jD4f2vs3NET-...,Nội thất đầy đủ,...,0879571234,104 triệu/m²,NaN,"2,6 tỷ",25 m²,"Ngõ 22 Phố Tôn Thất Tùng, Phường Khương Thượng...",SIÊU PHẨM ĐỐNG ĐA•NHỎ TIỀN•LÔ GÓC•25M2 CHỈ 2....,NaN,3,NaN
4,2 phòng,5 m,NaN,2 phòng,"Nhà mặt tiền Chiết Ga ( 567 Lê văn khương), p....",NaN,https://nha.chotot.com/mua-ban-nha-dat-quan-12...,Nhà mặt phố,https://cdn.chotot.com/kUbqhbsIInNyHMfb59Bt5rR...,Hoàn thiện cơ bản,...,0985239435,"35,09 triệu/m²",32 m,"5,65 tỷ",161 m²,"Hẻm 567 Đường Lê Văn Khương, Phường Hiệp Thành...","Nhà 5.15MX32M Mặt Tiền Chiết Ga, P.Hiệp Thành...",NaN,1,Nam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,4 phòng,7 m,NaN,3 phòng,🧯🧯🧯💣💣💣🧨🧨🧨 \r\nBán nhà góc hai Mặt tiền hẻm Xô ...,NaN,https://nha.chotot.com/mua-ban-quan-binh-thanh...,Nhà ngõ,https://cdn.chotot.com/XG39GORCPtN7sah2q8ewWOb...,NaN,...,0784732969,"84,56 triệu/m²",10 m,"5,75 tỷ",NaN,"Phường 25, Quận Bình Thạnh, Tp Hồ Chí Minh",VIP!! Góc 7x10.2.5 tâng.Xvnt p24 BT.gia 5.75 ...,NaN,3,NaN
2240,5 phòng,6 m,NaN,3 phòng,"Nhà rất đẹp , mai mẻ , iên tĩnh ….đảm bảo co t...",NaN,https://nha.chotot.com/mua-ban-quan-binh-tan-t...,Nhà ngõ,https://cdn.chotot.com/Oq0AvrASqEB5QKCTt9Aens-...,Nội thất cao cấp,...,0918288017,"61,11 triệu/m²",15 m,"5,5 tỷ",195 m²,"Phường An Lạc, Quận Bình Tân, Tp Hồ Chí MinhXe...",Nhà đep,NaN,2,Đông Bắc
2241,4 phòng,NaN,NaN,NaN,"Trệt, lầu, 84m2, 4 phòng ngủ, nhà cực rộng, cầ...",NaN,https://nha.chotot.com/mua-ban-quan-binh-thanh...,Nhà ngõ,https://cdn.chotot.com/XFqYuD-3BBoYKU1uSIpLq6V...,NaN,...,0911913139,"75,58 triệu/m²",NaN,"6,5 tỷ",170 m²,"Đường Trần Quý Cáp, Phường 11, Quận Bình Thạnh...",86m2 nhà 2 mặt tiền trước sau. số nhà mặt tiền,NaN,NaN,NaN
2242,3 phòng,NaN,NaN,3 phòng,Gia chủ cần bán nhà p7 q8 đường đình an tài vớ...,NaN,https://nha.chotot.com/mua-ban-quan-8-tp-ho-ch...,Nhà ngõ,https://cdn.chotot.com/P7lFUFE5yd-C567d12oKzGa...,Nội thất đầy đủ,...,0932222024,"48,19 triệu/m²",NaN,4 tỷ,NaN,"Đình An Tài, Phường 8, Quận 8, Tp Hồ Chí MinhX...","Bán nhà Diện tích lớn 5x17, hẻm xe hơi p7 quận 8",NaN,NaN,NaN


In [29]:
chotot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2244 entries, 0 to 2243
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Số phòng ngủ:                 2244 non-null   object
 1   Chiều ngang:                  1784 non-null   object
 2   Mã căn / Mã căn hộ:           14 non-null     object
 3   Số phòng vệ sinh:             1774 non-null   object
 4   Chi tiết                      2244 non-null   object
 5   Dự Án:                        115 non-null    object
 6   Link nguồn                    2244 non-null   object
 7   Loại hình nhà ở:              2243 non-null   object
 8   Ảnh nhà                       2244 non-null   object
 9   Tình trạng nội thất:          1126 non-null   object
 10  Đặc điểm nhà/đất:             589 non-null    object
 11  Giấy tờ pháp lý:              1790 non-null   object
 12  Người bán                     2244 non-null   object
 13  Diện tích đất:    

In [30]:
chotot['Giá đề xuất'] = chotot['Giá đề xuất'].apply(lambda x: normalize_price(str(x)))
chotot['Diện tích đất: '] = chotot['Diện tích đất: '].apply(lambda x: normalize_square(str(x)))   

In [31]:
chotot['Tên bài đăng'] = chotot['Tên bài đăng'].apply(lambda x: normalize_string(str(x)))   
chotot['Địa chỉ'] = chotot['Địa chỉ'].apply(lambda x: normalize_string(str(x)))  
chotot['Chi tiết'] = chotot['Chi tiết'].apply(lambda x: normalize_string(str(x)))   

In [32]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    #add data
    # BDS_DF_SAMPLE,tbl1,CHOTOT_DF_SAMPLE,tbl2=importdatasets()
    BDS_DF_SAMPLE = batdongsan.sample(50).astype(str)

    bds_header = list(batdongsan.columns.values)
    tbl1 = np.array(bds_header)

    CHOTOT_DF_SAMPLE = chotot.sample(50).astype(str)

    chotot_header = list(chotot.columns.values)
    tbl2 = np.array(chotot_header)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(BDS_DF_SAMPLE,CHOTOT_DF_SAMPLE,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)

    
    
main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#depre

Original Link ----- Link nguồn : 35.59682549233779
Tên người bán ----- Người bán : 67.05820638664179
Địa chỉ ----- Địa chỉ : 80.89303635827704
Mức giá ----- Giá đề xuất : 48.423543123543126
Ảnh ----- Ảnh nhà : 40.58134306735606
Số toilet ----- Số phòng vệ sinh:  : 39.147619047619045
Pháp lý ----- Giấy tờ pháp lý:  : 36.479656201849515
Tiêu đề ----- Tên bài đăng : 48.35280299921624
Mô tả ----- Chi tiết : 21.265120089800284
Hướng nhà ----- Hướng cửa chính:  : 45.06111111111111
Nội thất ----- Tình trạng nội thất:  : 32.00981929821093
Số điện thoại ----- Liên lạc : 50.6
Số phòng ngủ ----- Số phòng ngủ:  : 75.8
Diện tích ----- Diện tích đất:  : 80.26616161616161


In [33]:
bds_matching = pd.DataFrame(batdongsan_origin[['Original Link', 'Tiêu đề', 'Tên người bán', 'Số điện thoại', 'Địa chỉ', 'Mô tả', 'Mức giá', 'Diện tích', 'Ảnh', 'Hướng nhà', 'Số toilet', 'Số phòng ngủ', 'Pháp lý', 'Nội thất']])

In [34]:
bds_matching = bds_matching.dropna(subset = ['Ảnh'])

In [35]:
bds_matching['Mức giá'] = bds_matching['Mức giá'].apply(lambda x: normalize_price(str(x)))
bds_matching['Diện tích'] = bds_matching['Diện tích'].apply(lambda x: normalize_square(str(x)))   

In [36]:
chotot_matching = pd.read_csv("/content/Tich-Hop-Du-Lieu/DIProject/crawler/Cho-tot2.csv")

In [37]:
chotot_matching = chotot_matching.rename(columns={'Link nguồn': 'Original Link'})
chotot_matching = chotot_matching.rename(columns={'Người bán': 'Tên người bán'})
chotot_matching = chotot_matching.rename(columns={'Tên bài đăng': 'Tiêu đề'})
chotot_matching = chotot_matching.rename(columns={'Giá đề xuất':'Mức giá'})
chotot_matching = chotot_matching.rename(columns={'Ảnh nhà':'Ảnh'})
chotot_matching = chotot_matching.rename(columns={'Số phòng vệ sinh: ':'Số toilet'})
chotot_matching = chotot_matching.rename(columns={'Giấy tờ pháp lý: ':'Pháp lý'})
chotot_matching = chotot_matching.rename(columns={'Chi tiết':'Mô tả'})
chotot_matching = chotot_matching.rename(columns={'Hướng cửa chính: ':'Hướng nhà'})
chotot_matching = chotot_matching.rename(columns={'Tình trạng nội thất: ':'Nội thất'})
chotot_matching = chotot_matching.rename(columns={'Liên lạc':'Số điện thoại'})
chotot_matching = chotot_matching.rename(columns={'Diện tích đất: ':'Diện tích'})
chotot_matching = chotot_matching.rename(columns={'Số phòng ngủ: ':'Số phòng ngủ'})

In [38]:
chotot_matching = pd.DataFrame(chotot_matching[['Original Link', 'Tiêu đề', 'Tên người bán', 'Số điện thoại', 'Địa chỉ', 'Mô tả', 'Mức giá', 'Diện tích', 'Ảnh', 'Hướng nhà', 'Số toilet', 'Số phòng ngủ', 'Pháp lý', 'Nội thất']])

In [39]:
chotot_matching['Mức giá'] = chotot_matching['Mức giá'].apply(lambda x: normalize_price(str(x)))
chotot_matching['Diện tích'] = chotot_matching['Diện tích'].apply(lambda x: normalize_square(str(x)))   

BDS & Alonhadat

In [40]:
alonhadat = pd.read_csv("craw data/alonhadat.csv")

In [41]:
alonhadat = alonhadat.rename(columns={'Gia': 'Price'})
alonhadat = alonhadat.rename(columns={'Tên người bán': 'Người bán'})
alonhadat = alonhadat.rename(columns={'Địa chỉ': 'Addr'})
alonhadat = alonhadat.rename(columns={'Diện tích': 'Tổng diện tích'})


In [42]:
alonhadat.drop('Unnamed: 0', inplace=True, axis=1)

In [43]:
alonhadat

,Original Link,Người bán,Tiêu đề,Addr,Mô tả,Price,Tổng diện tích,Mã tin,Hướng,Loại tin,Đường trước nhà,Loại BDS,Pháp lý,Chiều ngang,Số lầu,Chiều dài,Số phòng ngủ,Ảnh
0,https://alonhadat.com.vn/ban-nha-mat-pho-trung...,Lê Đức Minh,"BÁN NHÀ MẶT PHỐ TRUNG KÍNH LỚN, CẦU GIẤY LÔ GÓ...","Từ Liêm Phố Trung Kính, Phường Trung Hòa, Quậ...",Chính chủ nhà nhờ bán Nhà Đất Mặt phố Trung Kí...,175 tỷ,360 m2,10994888,_,Cần bán,60m,Nhà mặt tiền,---,---,1,---,1,/files/properties/2022/6/26/images/010994888-5...
1,https://alonhadat.com.vn/ban-toa-nha-6-tang-ma...,Lê Đức Minh,BÁN TÒA NHÀ 6 TẦNG MẶT PHỐ TRẦN CUNG DT160M2 X...,"Từ Liêm Phố Trần Cung, Phường Quan Hoa, Quận ...","CHÍNH CHỦ NHỜ BÁN TÒA NHÀ 6 TẦNG THANG MÁY, DI...",34 tỷ,160 m2,11003935,_,Cần bán,30m,Nhà mặt tiền,---,---,6,---,6,/files/properties/2022/6/27/images/011003935-5...
2,https://alonhadat.com.vn/chinh-chu-ban-toa-nha...,Mr Thành,"CHÍNH CHỦ CẦN BÁN TOÀ NHÀ PHỐ NGHĨA TÂN: 140M,...","Đường Nghĩa Tân, Phường Nghĩa Tân, Quận Cầu G...","CHÍNH CHỦ CẦN BÁN TOÀ NHÀ PHỐ NGHĨA TÂN: 140M,...","29,9 tỷ",140 m2,8607391,Đông,Cần bán,10m,Nhà mặt tiền,Sổ hồng/ Sổ đỏ,10m,8,14m,18,/files/properties/2021/8/10/images/08607391-49...
3,https://alonhadat.com.vn/ban-toa-nha-vp-182m-8...,Mr Thành,"BÁN TOÀ NHÀ VP 182M, 8 TẦNG PHỐ LÊ TRỌNG TẤN, ...","Phố Lê Trọng Tấn, Phường Phương Mai, Quận Tha...","+ Hướng: Đông Bắc.\n+ Toà nhà 3 mặt thoáng, vỉ...",63 tỷ,182 m2,10622716,Đông Bắc,Cần bán,20m,Nhà mặt tiền,Sổ hồng/ Sổ đỏ,"5,6m",8,22m,6,/files/properties/2022/5/13/images/010622716-4...
4,https://alonhadat.com.vn/can-ban-gap-nha-40m2-...,Hoàng Hiếu,"Cần bán gấp nhà 40M2 x 5t, Nhà mới ở ngay, ngõ...","Đường Láng Hạ, Phường Láng Hạ, Quận Đống Đa...",NHÀ ĐẸP Ở NGAY - VỊ TRÍ TRUNG TÂM ĐỐNG ĐA. \n+...,"4,45 tỷ",40 m2,11011668,_,Cần bán,"2,5m",Nhà trong hẻm,---,---,5,---,3,/files/properties/2022/6/28/images/011011668-5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,https://alonhadat.com.vn/chinh-chu-ban-nha-mat...,Le Huy,CHÍNH CHỦ BÁN NHÀ MẶT PHỐ SỐ 71 ĐẠI LA - HBT ...,"71 Phố Đại La, Phường Trương Định, Quận Hai ...",Chính chủ cần bán căn nhà mặt phố số 71 Đại La...,"17,9 tỷ",55 m2,10819951,Tây Bắc,Cần bán,20m,Nhà mặt tiền,Sổ hồng/ Sổ đỏ,---,4,---,4,/files/properties/2022/6/6/images/010819951-04...
1636,https://alonhadat.com.vn/toi-chinh-chu-ban-nha...,Thắng,Bán nhà Mặt Phố số 17 Lương Văn Can Hoàn Kiếm ...,"Phố Lương Văn Can, Phường Hàng Đào, Quận Ho...",Bán nhà tầng 1 không nóc (có gác xép gần như l...,500 triệu / m2,60 m2,10771997,Đông Nam,Cần bán,20m,Nhà mặt tiền,Sổ hồng/ Sổ đỏ,"4,6m",2,---,2,/files/properties/2022/5/31/images/010771997-1...
1637,https://alonhadat.com.vn/mp-thanh-nhan-quan-ha...,Thắng,Bán nhà mặt phố số 20 Thanh Nhàn 50m2 x 3 tầng...,"Đường Thanh Nhàn, Phường Thanh Nhàn, Quận Ha...","Bán nhà mặt phố số 20 Thanh Nhàn, ngay gần bện...","20,2 tỷ",50 m2,10189978,_,Cần bán,12m,Nhà mặt tiền,Sổ hồng/ Sổ đỏ,4m,3,---,5,/files/properties/2022/3/23/images/010189978-1...
1638,https://alonhadat.com.vn/nha-mat-pho-my-dinh-k...,Thiều Thị Hải Vân,NHÀ MẶT PHỐ MỸ ĐÌNH- KINH DOANH- CHUNG CƯ- Ô T...,"Đường Mỹ Đình, Phường Mỹ Đình 1, Quận Nam Từ ...",🤩🤩🤩CCMN DÒNG TIỀN KHỦNG 🤩🤩\nMỹ Đình 100 m.7 tầ...,18 tỷ,100 m2,10786624,_,Cần bán,7m,Nhà mặt tiền,---,---,7,---,25,/files/properties/2022/6/1/images/010786624-21...


In [44]:
alonhadat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1640 entries, 0 to 1639
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Original Link    1640 non-null   object
 1   Người bán        1640 non-null   object
 2   Tiêu đề          1640 non-null   object
 3   Addr             1640 non-null   object
 4   Mô tả            1640 non-null   object
 5   Price            1640 non-null   object
 6   Tổng diện tích   1640 non-null   object
 7   Mã tin           1640 non-null   int64 
 8   Hướng            1640 non-null   object
 9   Loại tin         1640 non-null   object
 10  Đường trước nhà  1640 non-null   object
 11  Loại BDS         1640 non-null   object
 12  Pháp lý          1640 non-null   object
 13  Chiều ngang      1640 non-null   object
 14  Số lầu           1640 non-null   object
 15  Chiều dài        1640 non-null   object
 16  Số phòng ngủ     1640 non-null   object
 17  Ảnh              1640 non-null   

In [45]:
alonhadat['Số điện thoại'] = np.nan
alonhadat['Số Toilet'] = np.nan
alonhadat['Nội thất nhà'] = np.nan


In [46]:
def normalize_price_alo(x):   
    if 'tỷ' in x or 'triệu' in x:
      if (x[-3:]) == 'tỷ ':
        x = str(float(x[:-3].replace(',', '.'))*1000)
      x = x.replace('tỷ','')
      x = x.replace('triệu','')  
      x = x.replace(' ', '')
      try:
        x = float(x)
        if x > 0:
          return x
        else: 
          return str('Thỏa thuận')
      except:
        return str('Thỏa thuận')
    else:
      return str('Thỏa thuận')

In [47]:
alonhadat['Price'] = alonhadat['Price'].apply(lambda x: normalize_price_alo(str(x)))

In [48]:
def normalize_square_alo(x):
  x = x.split(' ')
  if x[1] != 'nan' and len(x[1]) > 0:
    x[1] = x[1].replace(',', '.')
    # return float(x[0])
    return x[1]

In [49]:
alonhadat['Tổng diện tích'] = alonhadat['Tổng diện tích'].apply(lambda x: normalize_square_alo(str(x)))   

In [50]:
alonhadat['Tiêu đề'] = alonhadat['Tiêu đề'].apply(lambda x: normalize_string(str(x)))   
alonhadat['Addr'] = alonhadat['Addr'].apply(lambda x: normalize_string(str(x)))  
alonhadat['Mô tả'] = alonhadat['Mô tả'].apply(lambda x: normalize_string(str(x)))   

In [51]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    #add data
    # BDS_DF_SAMPLE,tbl1,CHOTOT_DF_SAMPLE,tbl2=importdatasets()
    BDS_DF_SAMPLE = batdongsan.sample(50).astype(str)

    bds_header = list(batdongsan.columns.values)
    tbl1 = np.array(bds_header)

    ALONHADAT_DF_SAMPLE = alonhadat.sample(50).astype(str)

    alonhadat_header = list(alonhadat.columns.values)
    tbl2 = np.array(alonhadat_header)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(BDS_DF_SAMPLE,ALONHADAT_DF_SAMPLE,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)

    
    
main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#depre

Original Link ----- Original Link : 72.53298313603577
Tên người bán ----- Người bán : 68.5472109474965
Địa chỉ ----- Addr : 40.63173441683474
Mức giá ----- Price : 53.87745587745587
Ảnh ----- Ảnh : 58.58246451334695
Số toilet ----- Số Toilet : 50.0
Pháp lý ----- Pháp lý : 65.89989165922469
Tiêu đề ----- Tiêu đề : 75.59189501331633
Mô tả ----- Mô tả : 54.8026344797034
Hướng nhà ----- Hướng : 50.64285714285714
Nội thất ----- Nội thất nhà : 40.0
Số điện thoại ----- Số điện thoại : 50.0
Số phòng ngủ ----- Số phòng ngủ : 56.166666666666664
Diện tích ----- Tổng diện tích : 72.39285714285714


In [52]:
alonhadat_matching = pd.read_csv("craw data/alonhadat.csv")

In [53]:
alonhadat_matching = alonhadat_matching.rename(columns={'Gia': 'Price'})
alonhadat_matching = alonhadat_matching.rename(columns={'Tên người bán': 'Người bán'})
alonhadat_matching = alonhadat_matching.rename(columns={'Địa chỉ': 'Addr'})
alonhadat_matching = alonhadat_matching.rename(columns={'Diện tích': 'Tổng diện tích'})

In [54]:
alonhadat_matching['Số điện thoại'] = np.nan
alonhadat_matching['Số Toilet'] = np.nan
alonhadat_matching['Nội thất nhà'] = np.nan

In [55]:
alonhadat_matching = alonhadat_matching.rename(columns={'Người bán': 'Tên người bán'})
alonhadat_matching = alonhadat_matching.rename(columns={'Addr': 'Địa chỉ'})
alonhadat_matching = alonhadat_matching.rename(columns={'Price':'Mức giá'})
alonhadat_matching = alonhadat_matching.rename(columns={'Số Toilet':'Số toilet'})
alonhadat_matching = alonhadat_matching.rename(columns={'Hướng':'Hướng nhà'})
alonhadat_matching = alonhadat_matching.rename(columns={'Nội thất nhà':'Nội thất'})
alonhadat_matching = alonhadat_matching.rename(columns={'Tổng diện tích':'Diện tích'})

In [56]:
alonhadat_matching = pd.DataFrame(alonhadat_matching[['Original Link', 'Tiêu đề', 'Tên người bán', 'Số điện thoại', 'Địa chỉ', 'Mô tả', 'Mức giá', 'Diện tích', 'Ảnh', 'Hướng nhà', 'Số toilet', 'Số phòng ngủ', 'Pháp lý', 'Nội thất']])

In [57]:
alonhadat_matching['Mức giá'] = alonhadat_matching['Mức giá'].apply(lambda x: normalize_price_alo(str(x)))
alonhadat_matching['Diện tích'] = alonhadat_matching['Diện tích'].apply(lambda x: normalize_square_alo(str(x)))   

Batdongsan & Mogi

In [58]:
mogi = pd.read_csv("craw data/mogi.csv")

In [59]:
mogi = mogi.rename(columns={'Tên người bán': 'Seller'})
mogi = mogi.rename(columns={'Địa chỉ': 'Address'})
mogi = mogi.rename(columns={'Nhà tắm': 'Num Toilet'})

In [60]:
mogi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         736 non-null    int64  
 1   Original Link      736 non-null    object 
 2   Seller             736 non-null    object 
 3   Tiêu đề            736 non-null    object 
 4   Address            736 non-null    object 
 5   Mô tả              736 non-null    object 
 6   Giá                736 non-null    object 
 7   Diện tích sử dụng  634 non-null    object 
 8   Diện tích đất      675 non-null    object 
 9   Pháp lý            736 non-null    object 
 10  Ngày đăng          736 non-null    object 
 11  Mã BĐS             736 non-null    int64  
 12  Ảnh                736 non-null    object 
 13  Phòng ngủ          658 non-null    float64
 14  Num Toilet         658 non-null    float64
dtypes: float64(2), int64(2), object(11)
memory usage: 86.4+ KB


In [61]:
mogi['Số điện thoại'] = np.nan
mogi['Nội thất nhà'] = np.nan
mogi['Hướng chính'] = np.nan

In [62]:
mogi['Giá'] = mogi['Giá'].apply(lambda x: normalize_price(str(x)))
mogi['Diện tích đất'] = mogi['Diện tích đất'].apply(lambda x: normalize_square(str(x)))   

mogi['Tiêu đề'] = mogi['Tiêu đề'].apply(lambda x: normalize_string(str(x)))   
mogi['Address'] = mogi['Address'].apply(lambda x: normalize_string(str(x)))  
mogi['Mô tả'] = mogi['Mô tả'].apply(lambda x: normalize_string(str(x)))   
# mogi['Pháp lý'] = mogi['Pháp lý'].apply(lambda x: normalize_string(str(x))) 

In [63]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    corresponacy1.clear()
    #add data
    # BDS_DF_SAMPLE,tbl1,MOGI_DF_SAMPLE,tbl2=importdatasets()
    BDS_DF_SAMPLE = batdongsan.sample(50).astype(str)

    bds_header = list(batdongsan.columns.values)
    tbl1 = np.array(bds_header)

    MOGI_DF_SAMPLE = mogi.sample(50).astype(str)

    mogi_header = list(mogi.columns.values)
    tbl2 = np.array(mogi_header)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(BDS_DF_SAMPLE,MOGI_DF_SAMPLE,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)
    
    
main()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#depre

Original Link ----- Original Link : 72.46987697356676
Tên người bán ----- Seller : 28.347954981424817
Địa chỉ ----- Address : 39.017795075237466
Mức giá ----- Giá : 61.098634698634704
Ảnh ----- Ảnh : 61.32996202704665
Số toilet ----- Num Toilet : 42.28181818181818
Pháp lý ----- Pháp lý : 69.49221237047324
Tiêu đề ----- Tiêu đề : 77.52292995254153
Mô tả ----- Mô tả : 54.43534996372541
Hướng nhà ----- Hướng chính : 40.0
Nội thất ----- Nội thất nhà : 40.0
Số điện thoại ----- Số điện thoại : 50.0
Số phòng ngủ ----- Phòng ngủ : 49.481818181818184
Diện tích ----- Diện tích đất : 81.0


In [64]:
mogi_matching = pd.read_csv("craw data/mogi.csv")

In [65]:
mogi_matching = mogi_matching.rename(columns={'Tên người bán': 'Seller'})
mogi_matching = mogi_matching.rename(columns={'Địa chỉ': 'Address'})
mogi_matching = mogi_matching.rename(columns={'Nhà tắm': 'Num Toilet'})

In [66]:
mogi_matching['Số điện thoại'] = np.nan
mogi_matching['Nội thất nhà'] = np.nan
mogi_matching['Hướng chính'] = np.nan

In [67]:
mogi_matching = mogi_matching.rename(columns={'Link nguồn': 'Original Link'})
mogi_matching = mogi_matching.rename(columns={'Seller': 'Tên người bán'})
mogi_matching = mogi_matching.rename(columns={'Giá':'Mức giá'})
mogi_matching = mogi_matching.rename(columns={'Address':'Địa chỉ'})
mogi_matching = mogi_matching.rename(columns={'Num Toilet':'Số toilet'})
mogi_matching = mogi_matching.rename(columns={'Hướng chính':'Hướng nhà'})
mogi_matching = mogi_matching.rename(columns={'Nội thất nhà':'Nội thất'})
mogi_matching = mogi_matching.rename(columns={'Diện tích đất': 'Diện tích'})
mogi_matching = mogi_matching.rename(columns={'Phòng ngủ':'Số phòng ngủ'})

In [68]:
mogi_matching = pd.DataFrame(mogi_matching[['Original Link', 'Tiêu đề', 'Tên người bán', 'Số điện thoại', 'Địa chỉ', 'Mô tả', 'Mức giá', 'Diện tích', 'Ảnh', 'Hướng nhà', 'Số toilet', 'Số phòng ngủ', 'Pháp lý', 'Nội thất']])

In [69]:
mogi_matching['Mức giá'] = mogi_matching['Mức giá'].apply(lambda x: normalize_price(str(x)))
mogi_matching['Diện tích'] = mogi_matching['Diện tích'].apply(lambda x: normalize_square(str(x)))  

Concat data to one table

In [71]:
dataset = pd.concat([bds_matching, chotot_matching, alonhadat_matching, mogi_matching]) 

In [72]:
dataset = dataset.reset_index(drop=True)

In [73]:
dataset.describe().T

,count,unique,top,freq
Original Link,6846,6647,https://nha.chotot.com/mua-ban-thanh-pho-thuan...,2
Tiêu đề,6846,6589,Bán Gấp Nhà 5 Tầng Thang Máy Phố Hoàng Quốc Vi...,9
Tên người bán,6846,3658,Ngô Văn Minh,81
Số điện thoại,4470,3017,0901409378,15
Địa chỉ,6846,4180,"Dự án The Empire - Vinhomes Ocean Park 2, Xã L...",36
Mô tả,6846,6594,"- Nhà rất đẹp, có ô chờ thang máy, thoáng trướ...",9
Mức giá,6846,910,Thỏa thuận,751
Diện tích,6784.0,967.0,100.0,243.0
Ảnh,6846,6614,https://cdn.chotot.com/rEbZjVqF9vyLx53_ELRiWdx...,2
Hướng nhà,3109,13,_,1475


In [74]:
dataset.fillna('Chưa có thông tin', inplace=True)

In [75]:
dataset

,Original Link,Tiêu đề,Tên người bán,Số điện thoại,Địa chỉ,Mô tả,Mức giá,Diện tích,Ảnh,Hướng nhà,Số toilet,Số phòng ngủ,Pháp lý,Nội thất
0,https://batdongsan.com.vn/ban-dat-xa-tan-lap-1...,Bán 10 mẫu ngay mặt tiền đường dẫn cao tốc Ph...,Quách Hương,0979016016,"Xã Tân Lập, Hàm Thuận Nam, Bình Thuận",\r\n Đất ngay đường dẫn cao tốc...,Thỏa thuận,93252.0,https://file4.batdongsan.com.vn/resize/200x200...,Chưa có thông tin,Chưa có thông tin,Chưa có thông tin,Sổ đỏ/ Sổ hồng,Chưa có thông tin
1,https://batdongsan.com.vn/ban-dat-duong-quang-...,"Tôi không bán đất, tôi chỉ muốn khoe tôi có 2...",Quách Hương,0979016016,"Đường Quang Trung, Phường Tăng Nhơn Phú B, Quậ...",\r\n Sở hữu đất rộng ngay UBND ...,Thỏa thuận,238.0,https://file4.batdongsan.com.vn/resize/200x200...,Chưa có thông tin,Chưa có thông tin,Chưa có thông tin,Sổ đỏ/ Sổ hồng,Chưa có thông tin
2,https://batdongsan.com.vn/ban-dat-duong-dt-716...,Tôi không bán Đất. Tôi chỉ muốn khoe tôi có 3...,Quách Hương,0979016016,"Đường ĐT 716, Xã Hoà Thắng, Bắc Bình, Bình Thu...",\r\n Siêu phẩm đất view biển nằ...,Thỏa thuận,4169.0,https://file4.batdongsan.com.vn/resize/200x200...,Chưa có thông tin,Chưa có thông tin,Chưa có thông tin,Sổ đỏ/ Sổ hồng,Chưa có thông tin
3,https://batdongsan.com.vn/ban-dat-nen-du-an-du...,Hưng Thịnh mở bán 40 lô đất nền đô thị kiểu m...,Tuấn Trần,0901478123,"Dự án Richmond Quy Nhơn, Quy Nhơn, Bình Định",\r\n - Tập đoàn Hưng Thịnh mở b...,5300.0,100.0,https://file4.batdongsan.com.vn/resize/200x200...,Đông,Chưa có thông tin,Chưa có thông tin,Hợp đồng mua bán,Chưa có thông tin
4,https://batdongsan.com.vn/ban-nha-biet-thu-lie...,"Quỹ 5 căn góc liền kề, biệt thự trục chính, m...",Đào Tiến Minh,0348666658,"Dự án Mê Linh - Thanh Lâm - Đại Thịnh, Mê Linh...",\r\n Chính chủ hiện nay tôi có ...,7300.0,111.0,https://file4.batdongsan.com.vn/resize/200x200...,Chưa có thông tin,5 phòng,4 phòng,Chưa có thông tin,Nhà xây thô hoàn thiện mặt ngoài.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6841,https://mogi.vn/quan-ha-dong/mua-nha-hem-ngo/1...,1.8tỷ căn góc ngõ thông-KIA morning đỗ-Quận Hà...,LÊ LINH,Chưa có thông tin,"An Thắng, Phường Biên Giang, Quận Hà Đông, Hà...",Địa chỉ : Đường An Thắng-Phường Biên Giang- Qu...,1800.0,31.0,https://cloud.mogi.vn/images/2022/04/05/000/df...,Chưa có thông tin,3.0,3.0,Sổ đỏ,Chưa có thông tin
6842,https://mogi.vn/quan-long-bien/mua-nha-hem-ngo...,Ô tô tránh - Vỉa hè - Bán nhà Liền kề Thạch Bà...,Nguyễn Thị Nền,Chưa có thông tin,"Thạch Bàn, Phường Thạch Bàn, Quận Long Biên, ...",💋💥Ô TÔ TRÁNH - VỈA HÈ - MT 6.7m - BÁN NHÀ LIỀ...,11400.0,80.0,https://cloud.mogi.vn/images/2022/02/08/257/c7...,Chưa có thông tin,7.0,5.0,Không xác định,Chưa có thông tin
6843,https://mogi.vn/quan-long-bien/mua-nha-hem-ngo...,Bán nhà 38 m2 x 5 tầng ngõ 447 Ngọc Lâm quận L...,CHU MẠNH ĐỒNG,Chưa có thông tin,"Ngọc Lâm, Phường Ngọc Lâm, Quận Long Biên, Hà...",Bán nhà 38 m2 x 5 tầng ngõ 447 Ngọc Lâm quận L...,6500.0,38.0,https://cloud.mogi.vn/images/2021/11/21/317/64...,Chưa có thông tin,4.0,4.0,Sổ đỏ,Chưa có thông tin
6844,https://mogi.vn/quan-thanh-xuan/mua-nha-hem-ng...,"Mới đẹp, 5m ra ô tô-Quan Nhân, Thanh Xuân -DT ...",Lộc Bùi,Chưa có thông tin,"Quan Nhân, Phường Nhân Chính, Quận Thanh Xuân...","Bán nhà mới đẹp Phố Quan Nhân, Thanh Xuân, Nhâ...",4000.0,35.0,https://cloud.mogi.vn/images/2022/06/09/305/50...,Chưa có thông tin,4.0,3.0,Sổ đỏ,Chưa có thông tin


In [79]:
dataset.to_csv("DIProject/all_dataset/all_dataset_after_schema-matching.csv", index=False, header=True)